In [1]:
from tkinter import * 
import tkinter as tk
import tkinter.messagebox
from PIL import ImageTk, Image, ImageFont
import numpy as np
import cv2
import os
from tkinter import filedialog
from tkinter.filedialog import askopenfile


root = tk.Tk()
root.title("Uno Card Detection- Intelligent Sensing for Robotics")
canvas = tk.Canvas(root,height =700, width=800)
canvas.pack()
#canvas.pack(expand = YES, fill = BOTH)

# Loading a backgroud picture for the GUI
background_image = tk.PhotoImage(file='unobg2.png')
# creating an interface that look appling to the eye , also to serve as an exersice to add to my python coding knowledge
background_label = tk.Label(root, image=background_image)
background_label.place( relwidth=1, relheight=1)

# setting the display frame properties 
frame = tk.Frame(root, bg='black')
frame.place(relx=0.1, rely=0.1, relwidth=0.8, relheight=0.8)


app = Frame(root, bg="black")
app.place(relx=0.1, rely=0.23, relwidth=0.8, relheight=0.65)
#app.title("Card number and colour Detection")
# Create a label in the frame to display the videostrean or image 
lmain = Label(app)
lmain.grid()

# importing the tresholded training/matching files/images and assining the to a list
path = 'ImageReferences'
cards =[]
cardNames =[]
myCardList= os.listdir(path)
orb = cv2.ORB_create(nfeatures=2000)
Matchthreshold= 150
   

print('Total Cards Detected',len(myCardList))

for card in myCardList:
    currentCard = cv2.imread(f'{path}/{card}',0)
    cards.append(currentCard)
   
    #to display the filename without the extention
    cardNames.append(os.path.splitext(card)[0])
print(cardNames)

# this fuction is resposible for getting the card names 
def getCardName(indexVal):
    list0 = ['01', '02', '03', '04', '0g']
    list1 = ['11', '12', '13', '14'] 
    list2 = ['tw1', 'tw2', 'tw3', 'tw4']
    list3 = ['t1', 't2', 't3', 't4']
    list4 = ['f1', 'f2', 'f3', 'f4']
    list5 = ['5b', '5g', '5r', '5y'] 
    list6 = ['ss1', 'ss2', 'ss3', 'ss4']
    list7 = ['s1', 's2', 's3', 's4']
    list8 = ['8b', '8g', '8r', '8y']
    list9 = ['n1', 'n2', 'n3', 'n4'] 
    listSKP = ['noe', 'noe2']
    listD2 = ['p2', 'p22'] 
    listD4 = ['p4'] 
    listR = ['rev', 'rev2']
    listSH = ['sho']
    listWCA = ['spc']
      
    if cardNames[indexVal] in list0:
        cardValue = "ZERO"
    elif cardNames[indexVal] in list1:
        cardValue = "ONE"
    elif cardNames[indexVal] in list2:
        cardValue = "TWO"
    elif cardNames[indexVal] in list3:
        cardValue = "THREE"
    elif cardNames[indexVal] in list4:
        cardValue = "FOUR"
    elif cardNames[indexVal] in list5:
        cardValue = "FIVE"
    elif cardNames[indexVal] in list6:
        cardValue = "SIX"
    elif cardNames[indexVal] in list7:
        cardValue = "SEVEN"
    elif cardNames[indexVal] in list8:
        cardValue = "EIGHT"
    elif cardNames[indexVal] in list9:
        cardValue = "NINE"
    elif cardNames[indexVal] in listSKP:
        cardValue = "SKIP TURN"
    elif cardNames[indexVal] in listD2:
        cardValue = "DRAW TWO"
    elif cardNames[indexVal] in listD4:
        cardValue = "DRAW FOUR"
    elif cardNames[indexVal] in listWCA:
        cardValue = "WILD CARD ANY FACE"
    elif cardNames[indexVal] in listR:
        cardValue = "REVERSE"
    elif cardNames[indexVal] in listSH:
        cardValue = "SWOP HANDS"
    else:
        cardValue = "card index invalid"
    return cardValue
     
    
# this fuction uses  ord.detectAndCompute to dected and return a list of features 
def findCardIdentity(cards):
    findFeatureList=[]
    for findcard in cards:
        keypoints, findfeature= orb.detectAndCompute(findcard,None)
        findFeatureList.append(findfeature)
        
    return findFeatureList
findCardList= findCardIdentity(cards)
print(len(findCardList))

def featureExtracion(newCard,findFeatureList):
    cardKeyPoints, cardfeature= orb.detectAndCompute(newCard,None)
    bf = cv2.BFMatcher()
    matchedList=[]
    finalValue=-1
    
    try:
        for feature in findFeatureList:
            matches = bf.knnMatch(feature,cardfeature,k=2)
            good=[]

            for m,n in matches:
                if m.distance< 0.75* n.distance:
                    good.append([m])
            matchedList.append(len(good))        
            #print(len(good))
    except: 
        pass
    print(matchedList)
    if len(matchedList)!=0:
            if max(matchedList)> Matchthreshold:
                finalValue=  matchedList.index(max(matchedList))
                print (finalValue)
            
    return finalValue
# Capture from camera
cap = cv2.VideoCapture(0)

# function for video streaming
def video_stream():
    # assigning the filenames of the training/matching files to groups of card for lableling
   
    _, frame2 =cap.read()
    # reading in the frame and assigning its shape , height and width to hh and ww for the pixcel location
    hh, ww , _ = frame2.shape
    hsv_frame = cv2.cvtColor(frame2, cv2.COLOR_BGR2HSV)
    
    # GET CENTER POINT OF THE FRAME 
    # setting the center point to extract the card color from a pixel
    cx = int(hh / 2)
    cy = int(ww / 2)
    
    # Getting the  value of the pixel in the center of the frame. NB this can be any pixcel of your choice frame
    # I found the center works better when you hold up the image
    pixel_center = hsv_frame[cy, cx]
    
    # assigning the first value of the pixcel cent array to hue as in HSV
    hue_value =  pixel_center[0]
    
    # define Hue color ranges
    color = "Undefined"
    if hue_value < 5:
        color = "RED"
    elif hue_value < 22:
        color = "YELLOW"
    elif hue_value < 33:
        color = "YELLOW"
    elif hue_value < 78:
        color = "GREEN"
    elif hue_value < 131:
        color = "BLUE"
    elif hue_value < 170:
        color = "RED"
    else:
        color = "RED"
    
        
    # here im keeping the original stream frames for the display and userbility  
    OriginalStreamImg= cv2.cvtColor(frame2, cv2.COLOR_BGR2RGBA)
    # convert to gray in order to create thresholding
    gray = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)

    # threshold the grayscale image
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1]

    # remove white borders
    thresh = thresh[2:hh-2, 2:ww-2]
    thresh = cv2.copyMakeBorder(thresh, 2,2,2,2, cv2.BORDER_REPLICATE)
    #cv2.imshow("THRESH", thresh)
    
    # locating the pixel that will help extract the color feature
    pixel_center_bgr = frame2[cy, cx]
    
    # getting the value of the card color and displaying the color word in the same color as the card 
    b, g, r = int(pixel_center_bgr[0]), int(pixel_center_bgr[1]), int(pixel_center_bgr[2])
    #cv2.putText(OriginalStreamImg, color, (cx - 200, 100), 0, 3, (b, g, r), 5)
    
    # drawing a circle arrount the pixel that helps extract the color of the card for userbility
    cv2.circle(OriginalStreamImg, (cx,cy),5,(25,25,25),2)
    
    # passing the threshhold image to the feature matching function
    indexVal= featureExtracion(thresh,findCardList)
    
    #  getting the string name from the card file name found as a match 
    if indexVal !=-1:
        #calling the get card name value vs to using the card file name to display to the user
        cardValue = getCardName(indexVal)
        # concatinating the color and card value into one string for GUI display 
        displaytext= color+' '+ cardValue
        cv2.putText(OriginalStreamImg, displaytext, (cx - 200, 100), cv2.FONT_HERSHEY_COMPLEX,1, (b, g, r), 3)
    
    
    # Streaming the frame to the GUI  
    img = Image.fromarray(OriginalStreamImg)
    imgtk = ImageTk.PhotoImage(image=img)
    lmain.imgtk = imgtk
    lmain.configure(image=imgtk)
    lmain.after(1, video_stream)    
        
def ImUploadCallBack():
    
    global img
    f_types = [('Jpg Files', '*.jpg')]
    # getting the image path 
    filename = filedialog.askopenfilename(filetypes=f_types)
    
    # load image
    img = cv2.imread(filename,1)
       
    # resizing the image to a size that is close to the train/ matching images collected 
    img = cv2.resize(img , (600,945))
    hh, ww = img.shape[:2]
    
    hsv_frame = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    # convert to gray
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # threshold the grayscale image
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1]

    # remove white borders
    thresh = thresh[2:hh-2, 2:ww-2]
    thresh = cv2.copyMakeBorder(thresh, 2,2,2,2, cv2.BORDER_REPLICATE)
    
    #cv2.imshow("STREAM2", thresh) uncoment to view the treshhold image used in the matching
    
    #hsv_frame = cv2.cvtColor(frame2, cv2.COLOR_BGR2HSV)
    
    # GET CENTER POINT OF THE FRAME 
    # setting the center point to extract the card color from a pixel
    cx = int(hh -520)
    cy = int(ww - 340)
    
    # Getting the  value of the pixel in the center of the frame. NB this can be any pixcel of your choice frame
    # I found the center works better when you hold up the image
    pixel_center = hsv_frame[cy, cx]
    
    # assigning the first value of the pixcel cent array to hue as in HSV
    hue_value =  pixel_center[0]
    
    # define Hue color ranges
    color = "Undefined"
    if hue_value < 5:
        color = "RED"
    elif hue_value < 22:
        color = "YELLOW"
    elif hue_value < 33:
        color = "YELLOW"
    elif hue_value < 78:
        color = "GREEN"
    elif hue_value < 131:
        color = "BLUE"
    elif hue_value < 170:
        color = "RED"
    else:
        color = "RED"
       
    img=Image.open(filename)
    img_resized=img.resize((300,450)) # new width & height for gui display
    img=ImageTk.PhotoImage(img_resized)
    
    # Displaying the uploaded image to the GUI
    lmain.imgtk = img
    lmain.configure(image=img)
    
    # calling the the feature extration method that uses the k nearst nabour approch ,
    # This method return the index of the matched image index in the card list or folder 
    indexVal= featureExtracion(thresh,findCardList)
    
    if indexVal !=-1:
        cardValue = getCardName(indexVal)
        #cv2.putText(img,cardNames[indexVal],(50,50),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,255),3)
        
        #concatinating the color and card value for display 
        displaytext= color+' '+ cardValue
        tkinter.messagebox.showinfo("Detected Card Values ", displaytext)
    
# top two pink buttons for the video stream and upload image options           
vs = tkinter.Button(frame, text ="Video stream",bg='pink',anchor=CENTER, command = video_stream)
vs.place(relx=0,rely=0, relwidth=0.5,relheight=0.15)

imUpload= tkinter.Button(frame, text ="Upload Image" ,bg='pink',anchor=CENTER, command = ImUploadCallBack)
imUpload.place(relx=0.5,rely=0, relwidth=0.5,relheight=0.15)


root.mainloop()

Total Cards Detected 50
['01', '02', '03', '04', '0g', '11', '12', '13', '14', '5b', '5g', '5r', '5y', '8b', '8g', '8r', '8y', 'f1', 'f2', 'f3', 'f4', 'n1', 'n2', 'n3', 'n4', 'noe', 'noe2', 'p2', 'p22', 'p4', 'rev', 'rev2', 's1', 's2', 's3', 's4', 'sho', 'spc', 'ss1', 'ss2', 'ss3', 'ss4', 't1', 't2', 't3', 't4', 'tw1', 'tw2', 'tw3', 'tw4']
50
[68, 64, 77, 44, 17, 12, 25, 55, 26, 21, 22, 81, 74, 13, 5, 65, 96, 57, 67, 48, 63, 91, 24, 72, 58, 66, 54, 78, 87, 85, 490, 79, 20, 111, 63, 48, 31, 61, 86, 20, 31, 17, 38, 14, 20, 95, 17, 28, 86, 30]
30
